In [70]:
import pandas as pd

df_check = pd.DataFrame(

[['2025-2026','2025-08-17','Premier League','1','1', '0-1',None,90,0,0,None,0.0],
['2025-2026','2025-08-24','Premier League','2','10','1-1',None,90,0,0,None,0.0],
['2025-2026','2025-08-24','Premier League','2','10','1-1',None,90,0,0,None,0.0]], columns=['Season', 'Date','Comp', 'Round', 'Opponent', 'Result', 'Pos', 'Min', 'Gls', 'Ast', 'Points', 'Elo'])

In [71]:
df_check.drop_duplicates(subset = ['Season', 'Date'], keep='last', inplace=True)

In [72]:
df_check

,Season,Date,Comp,Round,Opponent,Result,Pos,Min,Gls,Ast,Points,Elo
0,2025-2026,2025-08-17,Premier League,1,1,0-1,None,90,0,0,None,0.0
2,2025-2026,2025-08-24,Premier League,2,10,1-1,None,90,0,0,None,0.0


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import aiohttp
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
import aiohttp
from fpl import FPL

async def get_all_fpl_player_names():
    async with aiohttp.ClientSession() as session:
        fpl = FPL(session)
        players = await fpl.get_players()
        names = [f"{player.first_name} {player.second_name}" for player in players]
        return names

# In a notebook cell, use:
names = await get_all_fpl_player_names()

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1016)>

In [236]:
import os
import pandas as pd

def get_fbref_ids_selenium(names, save_path="fbref_ids2.csv"):
    """
    Given a list of player names, use Selenium to search FBref and extract the player ID from the redirected URL.
    Skips names already processed and saves progress to a CSV file.
    """
    # Load existing results if file exists
    if os.path.exists(save_path):
        df_existing = pd.read_csv(save_path)
        done_names = set(df_existing['name'])
        results = df_existing.values.tolist()
    else:
        done_names = set()
        results = []

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    for name in names:
        try:
            if name in done_names:
                continue
            search_url = f"https://fbref.com/en/search/search.fcgi?search={requests.utils.quote(name)}"
            driver.get(search_url)
            final_url = driver.current_url
            parts = final_url.split("/")
            found_id = None
            if "players" in parts:
                idx = parts.index("players")
                if idx + 1 < len(parts):
                    found_id = parts[idx + 1]
            results.append([name, found_id])
            # Save after each lookup
            pd.DataFrame(results, columns=["name", "id"]).to_csv(save_path, index=False, encoding='utf-8')
        except Exception as e:
            print(f"Error processing {name}: {e}")
            results.append([name, None])
    driver.quit()
    return results

# Example usage
ids = get_fbref_ids_selenium(names)


In [ ]:


id = '79300479'
url = f'https://fbref.com/en/players/{id}/matchlogs/2024-2025/Martin-Odegaard-Match-Logs'

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(url)

soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

# Extract subheaders (second row of headers) and use them as DataFrame columns
table = soup.find('table', id='matchlogs_all')
if table:
    rows = table.find_all('tr')
    # Use the second row of headers (subheaders)
    subheaders = [header.text for header in rows[1].find_all('th')]
    # Insert 'Date' as the first column
    print('Subheaders:', subheaders)
    # Extract the first column (Date) along with the rest of the cells
    cells = [rows[3].find('th').text] + [cell.text for cell in rows[3].find_all('td')]
    print(cells)
    data = []
    for row in rows[2:]:
        print('herer')
        cells = [row.find('th').text] + [cell.text for cell in row.find_all('td')]
        if len(cells) == len(subheaders):
            data.append(cells)
        else:
            print(f'Skipped row with {len(cells)} columns (expected {len(subheaders)})')
    df = pd.DataFrame(data, columns=subheaders)
    display(df.head())
else:
    print('Match logs table not found.')

In [240]:
import os

years = ['2022-2023', '2023-2024', '2024-2025']
df_ids = pd.read_csv("fbref_ids2.csv",encoding='utf-8')

output_folder = "fbref_data"
os.makedirs(output_folder, exist_ok=True)

for _, row in df_ids.iterrows():
    try:
        player_id = row['id']
        player_name = row['name']
        filename = os.path.join(output_folder, f"{player_name.replace(' ', '_')}.csv")
        if os.path.exists(filename):
            print(f"CSV already exists for {player_name}, skipping.")
            continue
        player_data = []
        if pd.notnull(player_id):
            for year in years:
                driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
                url = f'https://fbref.com/en/players/{player_id}/matchlogs/{year}/{player_name.replace(" ", "-")}-Match-Logs'
                driver.get(url)
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                driver.quit()
                table = soup.find('table', id='matchlogs_all')
                if table:
                    rows = table.find_all('tr')
                    subheaders = [header.text for header in rows[1].find_all('th')]
                    for row in rows[2:]:
                        cells = [row.find('th').text] + [cell.text for cell in row.find_all('td')]
                        if len(cells) == len(subheaders):
                            player_data.append([year] + cells)
                else:
                    print(f'Match logs table not found for {player_name} ({year})')
            # Save to CSV if data exists
            if player_data:
                columns = ['Season'] + subheaders
                df_player = pd.DataFrame(player_data, columns=columns)
                df_player.to_csv(filename, index=False)
    except Exception as e:
        print(f"Error processing {player_name}: {e}")


CSV already exists for David Raya Martín, skipping.
CSV already exists for Kepa Arrizabalaga Revuelta, skipping.
CSV already exists for Tommy Setford, skipping.
CSV already exists for Gabriel Magalhães, skipping.
CSV already exists for William Saliba, skipping.
CSV already exists for Riccardo Calafiori, skipping.
CSV already exists for Jurriën Timber, skipping.
CSV already exists for Jakub Kiwior, skipping.
CSV already exists for Myles Lewis-Skelly, skipping.
CSV already exists for Benjamin White, skipping.
CSV already exists for Oleksandr Zinchenko, skipping.
CSV already exists for Brayden Clarke, skipping.
CSV already exists for Maldini Kacurri, skipping.
CSV already exists for Josh Nichols, skipping.
CSV already exists for Bukayo Saka, skipping.
CSV already exists for Martin Ødegaard, skipping.
CSV already exists for Noni Madueke, skipping.
CSV already exists for Gabriel Martinelli Silva, skipping.
CSV already exists for Leandro Trossard, skipping.
CSV already exists for Declan Rice

In [231]:
import os

years = ['2022-2023', '2023-2024', '2024-2025']
df_ids = ['e342ad68']

output_folder = "fbref_data"
os.makedirs(output_folder, exist_ok=True)

for _, row in df_ids.iterrows():
    try:
        player_id = row['id']
        player_name = row['name']
        filename = os.path.join(output_folder, f"{player_name.replace(' ', '_')}.csv")
        if os.path.exists(filename):
            print(f"CSV already exists for {player_name}, skipping.")
            continue
        player_data = []
        if pd.notnull(player_id):
            for year in years:
                url = f'https://fbref.com/en/players/{player_id}/matchlogs/{year}/{player_name.replace(" ", "-")}-Match-Logs'
                driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
                driver.get(url)
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                driver.quit()
                table = soup.find('table', id='matchlogs_all')
                if table:
                    rows = table.find_all('tr')
                    subheaders = [header.text for header in rows[1].find_all('th')]
                    for row in rows[2:]:
                        cells = [row.find('th').text] + [cell.text for cell in row.find_all('td')]
                        if len(cells) == len(subheaders):
                            player_data.append([year] + cells)
                else:
                    print(f'Match logs table not found for {player_name} ({year})')
            # Save to CSV if data exists
            if player_data:
                columns = ['Season'] + subheaders
                df_player = pd.DataFrame(player_data, columns=columns)
                df_player.to_csv(filename, index=False)
    except Exception as e:
        print(f"Error processing {player_name}: {e}")


AttributeError: 'list' object has no attribute 'iterrows'

In [170]:
import re

df_player = pd.read_csv("./fbref_data/Erling_Haaland.csv")
df_player = df_player[df_player['Date'].notna() & (df_player['Date'].str.strip() != "")]
df_player = df_player[['Season', 'Date', 'Comp', 'Round', 'Opponent', 'Result', 'Pos', 'Min', 'Gls', 'Ast']]
df_player['Result'] = df_player['Result'].apply(lambda x: re.sub(r"\(\d+\)", "", x))
df_player['GoalsConceded'] = df_player['Result'].apply(
    lambda x: x.replace("W ", "")
               .replace("L ", "")
               .replace("D ", "")
               .replace("–", "-")
               .strip().split("-")[1]
)
df_player['GoalsScored'] = df_player['Result'].apply(
    lambda x: x.replace("W ", "")
               .replace("L ", "")
               .replace("D ", "")
               .replace("–", "-")
               .strip().split("-")[0]
)
df_player['CleanSheet'] = df_player['GoalsConceded'].astype(int).apply(lambda x: 1 if x == 0 else 0)
df_player['GoalDifference'] = df_player['GoalsScored'].astype(int) - df_player['GoalsConceded'].astype(int)
df_player.head()

,Season,Date,Comp,Round,Opponent,Result,Pos,Min,Gls,Ast,GoalsConceded,GoalsScored,CleanSheet,GoalDifference
0,2022-2023,2022-06-02,UEFA Nations League,League B,rs Serbia,W 1–0,FW,66,1,0,0,1,1,1
1,2022-2023,2022-06-05,UEFA Nations League,League B,se Sweden,W 2–1,FW,74,2,0,1,2,0,1
2,2022-2023,2022-06-09,UEFA Nations League,League B,si Slovenia,D 0–0,FW,90,0,0,0,0,1,0
3,2022-2023,2022-06-12,UEFA Nations League,League B,se Sweden,W 3–2,FW,88,2,1,2,3,0,1
4,2022-2023,2022-07-30,Community Shield,FA Community Shield,Liverpool,L 1–3,FW,90,0,0,3,1,0,-2


In [171]:
def get_player_position(df_player):
    positions = df_player['Pos'].dropna().astype(str)
    attacker_count = positions.str.count(r'(S|F|W)').sum()
    midfielder_count = positions.str.count(r'M').sum()
    defender_count = positions.str.count(r'B').sum()
    keeper_count = positions.str.count(r'G').sum()
    counts = {
        'Attacker': attacker_count,
        'Midfielder': midfielder_count,
        'Defender': defender_count,
        'Keeper': keeper_count
    }
    main_position = max(counts, key=counts.get)
    return main_position

get_player_position(df_player)

'Attacker'

In [177]:
def midfielder_points(df_player):
    """
    Calculate points for a midfielder based on goals and assists.
    Points: 5 for a goal, 3 for an assist.
    """
    df_player['Points'] = df_player['Gls'].astype(int) * 5 + df_player['Ast'].astype(int) * 3
    return df_player[['Season', 'Date', 'Comp', 'Round', 'Opponent', 'Result', 'Pos', 'Min', 'Gls', 'Ast', 'Points']]

def attacker_points(df_player):
    """
    Calculate points for an attacker based on goals, assists, and clean sheets.
    Points: 5 for a goal, 3 for an assist, 1 for a clean sheet.
    """
    df_player['CleanSheets'] = df_player['GoalsConceded'].astype(int).apply(lambda x: 1 if x == 0 else 0)
    df_player['GoalDifference'] = df_player['GoalsScored'].astype(int) - df_player['GoalsConceded'].astype(int)
    df_player['Points'] = (df_player['Gls'].astype(int) * 5 +
                           df_player['Ast'].astype(int) * 3 +
                           df_player['CleanSheets'].astype(int))
    return df_player[['Season', 'Date', 'Comp', 'Round', 'Opponent', 'Result', 'Pos', 'Min', 'Gls', 'Ast', 'Points']]


def defender_points(df_player):
    """
    Calculate points for a defender based on goals, assists, and clean sheets.
    Points: 6 for a goal, 3 for an assist, 1 for a clean sheet.
    """
    df_player['Points'] = (df_player['Gls'].astype(int) * 6 +
                           df_player['Ast'].astype(int) * 3 +
                           df_player['CleanSheets'].astype(int))
    return df_player[['Season', 'Date', 'Comp', 'Round', 'Opponent', 'Result', 'Pos', 'Min', 'Gls', 'Ast', 'Points']]

def keeper_points(df_player):
    """
    Calculate points for a goalkeeper based on clean sheets and saves.
    Points: 4 for a clean sheet, 1 for every 3 saves.
    """
    df_player['Points'] = (df_player['CleanSheets'].astype(int) * 4 +
                           df_player['Saves'].astype(int) // 3)
    return df_player[['Season', 'Date', 'Comp', 'Round', 'Opponent', 'Result', 'Pos', 'Min', 'Gls', 'Ast', 'Points']]

In [178]:
df_player = attacker_points(df_player)

In [179]:
df_player = df_player.reset_index(drop=True)


In [ ]:
def player_elo(df_player, k=20):
    initial_elo = 1500.0  # Starting Elo rating
    df_player = df_player.reset_index(drop=True) # Ensure DataFrame is indexed correctly
    df_player['Elo'] = 0.0  # Initialize Elo column
    df_player.at[0, 'Elo'] = initial_elo  # Only the first match starts at 1500
    for i in range(1, len(df_player)):
        Ra = df_player.loc[i-1, 'Elo']
        Pa = df_player.loc[i-1, 'Points']
        # Expected score based on Elo (normalized to points scale)
        E_a = round( k/(1 + 10**(initial_elo/Ra)), 2)
        # Update Elo
        if Pa >= E_a:
            df_player.at[i, 'Elo'] = round((Ra + k * (Pa - E_a)), 3)
        else:
            # Lose points: penalize by k * (E_a * Pa - Pa)
            df_player.at[i, 'Elo'] = round((Ra + k * (Pa - E_a)), 3)
        last_elo = df_player.at[i, 'Elo']
    return last_elo

df_player_elo = player_elo(df_player)

Match 1: Ra=1500.0, Pa=6, E_a=1.82
Match 2: Ra=1583.6, Pa=10, E_a=2.03
Match 3: Ra=1743.0, Pa=1, E_a=2.42
Match 4: Ra=1714.6, Pa=13, E_a=2.35
Match 5: Ra=1927.6, Pa=0, E_a=2.86
Match 6: Ra=1870.4, Pa=11, E_a=2.73
Match 7: Ra=2035.8, Pa=4, E_a=3.1
Match 8: Ra=2053.8, Pa=5, E_a=3.14
Match 9: Ra=2091.0, Pa=15, E_a=3.22
Match 10: Ra=2326.6, Pa=16, E_a=3.69
Match 11: Ra=2572.8, Pa=5, E_a=4.14
Match 12: Ra=2590.0, Pa=11, E_a=4.17
Match 13: Ra=2726.6, Pa=5, E_a=4.4
Match 14: Ra=2738.6, Pa=6, E_a=4.42
Match 15: Ra=2770.2, Pa=5, E_a=4.47
Match 16: Ra=2780.8, Pa=0, E_a=4.48
Match 17: Ra=2691.2, Pa=21, E_a=4.34
Match 18: Ra=3024.4, Pa=11, E_a=4.84
Match 19: Ra=3147.6, Pa=6, E_a=5.0
Match 20: Ra=3167.6, Pa=0, E_a=5.03
Match 21: Ra=3067.0, Pa=10, E_a=4.9
Match 22: Ra=3169.0, Pa=1, E_a=5.03
Match 23: Ra=3088.4, Pa=5, E_a=4.93
Match 24: Ra=3089.8, Pa=0, E_a=4.93
Match 25: Ra=2991.2, Pa=5, E_a=4.79
Match 26: Ra=2995.4, Pa=10, E_a=4.8
Match 27: Ra=3099.4, Pa=5, E_a=4.94
Match 28: Ra=3100.6, Pa=1, E_a=4

In [230]:
df_player_elo

np.float64(2780.2)

In [228]:
df_player_elo.to_csv("elo.csv", index=False)

In [140]:
import aiohttp
from fpl import FPL

async def get_player_cost(player_name):
    async with aiohttp.ClientSession() as session:
        fpl = FPL(session)
        players = await fpl.get_players()
        for player in players:
            full_name = f"{player.first_name} {player.second_name}"
            if full_name.lower() == player_name.lower():
                print(f"{full_name} costs £{player.now_cost / 10}m")
                return player.now_cost / 10
        print(f"Player '{player_name}' not found.")
        return None

# Example usage:
cost = await get_player_cost("Erling Haaland")

Erling Haaland costs £14.0m


In [163]:
import pandas as pd
from pulp import LpProblem, LpVariable, lpSum, LpMaximize, LpBinary, LpStatus, value

def select_top_n_squads(csv_path="elo_ratings.csv", budget=85, attacker_count=3, midfielder_count=4, defender_count=3, keeper_count=1, top_n=3):
    df = pd.read_csv(csv_path)
    gks = df[df['Position'] == 'Keeper'].nlargest(10, 'Elo')
    defs = df[df['Position'] == 'Defender'].nlargest(40, 'Elo')
    mids = df[df['Position'] == 'Midfielder'].nlargest(40, 'Elo')
    atts = df[df['Position'] == 'Attacker'].nlargest(40, 'Elo')
    squad_df = pd.concat([gks, defs, mids, atts], ignore_index=True)

    squads = []
    used_combinations = set()

    for _ in range(top_n):
        choices = [LpVariable(f"player_{i}", cat=LpBinary) for i in range(len(squad_df))]
        prob = LpProblem("FPL_Squad_Selection", LpMaximize)
        prob += lpSum([choices[i] * squad_df.loc[i, "Elo"] for i in range(len(squad_df))])
        prob += lpSum([choices[i] * squad_df.loc[i, "Cost"] for i in range(len(squad_df))]) <= budget
        prob += lpSum([choices[i] for i in range(len(squad_df)) if squad_df.loc[i, "Position"] == "Keeper"]) == keeper_count
        prob += lpSum([choices[i] for i in range(len(squad_df)) if squad_df.loc[i, "Position"] == "Defender"]) == defender_count
        prob += lpSum([choices[i] for i in range(len(squad_df)) if squad_df.loc[i, "Position"] == "Midfielder"]) == midfielder_count
        prob += lpSum([choices[i] for i in range(len(squad_df)) if squad_df.loc[i, "Position"] == "Attacker"]) == attacker_count

        # Exclude previous squads
        for prev in used_combinations:
            prob += lpSum([choices[i] for i in prev]) <= len(prev) - 1

        prob.solve()
        selected = [i for i in range(len(squad_df)) if choices[i].varValue == 1]
        if not selected or tuple(selected) in used_combinations:
            break
        used_combinations.add(tuple(selected))
        result = squad_df.iloc[selected]
        squads.append(result)

    for idx, squad in enumerate(squads, 1):
        print(f"\nSquad {idx}:")
        print(f"Total Cost: {squad['Cost'].sum()} million")
        print(f"Total Elo: {squad['Elo'].sum()}")
        print(squad)
    return squads

# Example usage:
select_top_n_squads("elo_ratings.csv", budget=85, top_n=3)



Squad 1:
Total Cost: 84.5 million
Total Elo: 46507.1
                 Player    Position     Elo  Cost
0       Jordan_Pickford      Keeper  3217.0   5.5
10      Maxim_De_Cuyper    Defender  3313.1   4.5
11   Daniel_Muñoz_Mejía    Defender  3281.3   5.5
12  Pedro_Porro_Sauceda    Defender  2940.5   5.5
50           Cody_Gakpo  Midfielder  4700.3   7.5
51          Bukayo_Saka  Midfielder  4458.3  10.0
52        Florian_Wirtz  Midfielder  4108.1   8.5
54     Leandro_Trossard  Midfielder  3824.4   7.0
90       Erling_Haaland    Attacker  6377.1  14.0
91      Viktor_Gyökeres    Attacker  5828.4   9.0
92       Benjamin_Sesko    Attacker  4458.6   7.5

Squad 2:
Total Cost: 84.0 million
Total Elo: 46483.9
                 Player    Position     Elo  Cost
1           André_Onana      Keeper  3193.8   5.0
10      Maxim_De_Cuyper    Defender  3313.1   4.5
11   Daniel_Muñoz_Mejía    Defender  3281.3   5.5
12  Pedro_Porro_Sauceda    Defender  2940.5   5.5
50           Cody_Gakpo  Midfielder  4700.

[                 Player    Position     Elo  Cost
 0       Jordan_Pickford      Keeper  3217.0   5.5
 10      Maxim_De_Cuyper    Defender  3313.1   4.5
 11   Daniel_Muñoz_Mejía    Defender  3281.3   5.5
 12  Pedro_Porro_Sauceda    Defender  2940.5   5.5
 50           Cody_Gakpo  Midfielder  4700.3   7.5
 51          Bukayo_Saka  Midfielder  4458.3  10.0
 52        Florian_Wirtz  Midfielder  4108.1   8.5
 54     Leandro_Trossard  Midfielder  3824.4   7.0
 90       Erling_Haaland    Attacker  6377.1  14.0
 91      Viktor_Gyökeres    Attacker  5828.4   9.0
 92       Benjamin_Sesko    Attacker  4458.6   7.5,
                  Player    Position     Elo  Cost
 1           André_Onana      Keeper  3193.8   5.0
 10      Maxim_De_Cuyper    Defender  3313.1   4.5
 11   Daniel_Muñoz_Mejía    Defender  3281.3   5.5
 12  Pedro_Porro_Sauceda    Defender  2940.5   5.5
 50           Cody_Gakpo  Midfielder  4700.3   7.5
 51          Bukayo_Saka  Midfielder  4458.3  10.0
 52        Florian_Wirtz  Midf

In [166]:
import pandas as pd
from pulp import LpProblem, LpVariable, lpSum, LpMaximize, LpBinary

def select_top_n_squads(csv_path="elo_ratings.csv", budget=85, attacker_count=3, midfielder_count=4, defender_count=3, keeper_count=1, top_n=3):
    df = pd.read_csv(csv_path)
    gks = df[df['Position'] == 'Keeper'].nlargest(10, 'Elo')
    defs = df[df['Position'] == 'Defender'].nlargest(40, 'Elo')
    mids = df[df['Position'] == 'Midfielder'].nlargest(40, 'Elo')
    atts = df[df['Position'] == 'Attacker'].nlargest(40, 'Elo')
    squad_df = pd.concat([gks, defs, mids, atts], ignore_index=True)

    squads = []
    used_players = set()

    for _ in range(top_n):
        # Only use players not already selected
        available_idx = [i for i in range(len(squad_df)) if squad_df.loc[i, "Player"] not in used_players]
        if len(available_idx) < (keeper_count + defender_count + midfielder_count + attacker_count):
            break

        choices = [LpVariable(f"player_{i}", cat=LpBinary) for i in available_idx]
        prob = LpProblem("FPL_Squad_Selection", LpMaximize)
        prob += lpSum([choices[j] * squad_df.loc[available_idx[j], "Elo"] for j in range(len(available_idx))])
        prob += lpSum([choices[j] * squad_df.loc[available_idx[j], "Cost"] for j in range(len(available_idx))]) <= budget
        prob += lpSum([choices[j] for j in range(len(available_idx)) if squad_df.loc[available_idx[j], "Position"] == "Keeper"]) == keeper_count
        prob += lpSum([choices[j] for j in range(len(available_idx)) if squad_df.loc[available_idx[j], "Position"] == "Defender"]) == defender_count
        prob += lpSum([choices[j] for j in range(len(available_idx)) if squad_df.loc[available_idx[j], "Position"] == "Midfielder"]) == midfielder_count
        prob += lpSum([choices[j] for j in range(len(available_idx)) if squad_df.loc[available_idx[j], "Position"] == "Attacker"]) == attacker_count

        prob.solve()
        selected = [available_idx[j] for j in range(len(available_idx)) if choices[j].varValue == 1]
        if not selected:
            break
        used_players.update(squad_df.loc[selected, "Player"])
        result = squad_df.iloc[selected]
        squads.append(result)

    for idx, squad in enumerate(squads, 1):
        print(f"\nSquad {idx}:")
        print(f"Total Cost: {squad['Cost'].sum()} million")
        print(f"Total Elo: {squad['Elo'].sum()}")
        print(squad)
    return squads

# Example usage:
select_top_n_squads("elo_ratings.csv", budget=85, top_n=4)


Squad 1:
Total Cost: 84.5 million
Total Elo: 46507.1
                 Player    Position     Elo  Cost
0       Jordan_Pickford      Keeper  3217.0   5.5
10      Maxim_De_Cuyper    Defender  3313.1   4.5
11   Daniel_Muñoz_Mejía    Defender  3281.3   5.5
12  Pedro_Porro_Sauceda    Defender  2940.5   5.5
50           Cody_Gakpo  Midfielder  4700.3   7.5
51          Bukayo_Saka  Midfielder  4458.3  10.0
52        Florian_Wirtz  Midfielder  4108.1   8.5
54     Leandro_Trossard  Midfielder  3824.4   7.0
90       Erling_Haaland    Attacker  6377.1  14.0
91      Viktor_Gyökeres    Attacker  5828.4   9.0
92       Benjamin_Sesko    Attacker  4458.6   7.5

Squad 2:
Total Cost: 77.0 million
Total Elo: 37445.1
                Player    Position     Elo  Cost
1          André_Onana      Keeper  3193.8   5.0
13     Virgil_van_Dijk    Defender  2747.5   6.0
14      Joško_Gvardiol    Defender  2657.2   6.0
15        Fabian_Schär    Defender  2470.9   5.5
53         Cole_Palmer  Midfielder  3831.7  10.

[                 Player    Position     Elo  Cost
 0       Jordan_Pickford      Keeper  3217.0   5.5
 10      Maxim_De_Cuyper    Defender  3313.1   4.5
 11   Daniel_Muñoz_Mejía    Defender  3281.3   5.5
 12  Pedro_Porro_Sauceda    Defender  2940.5   5.5
 50           Cody_Gakpo  Midfielder  4700.3   7.5
 51          Bukayo_Saka  Midfielder  4458.3  10.0
 52        Florian_Wirtz  Midfielder  4108.1   8.5
 54     Leandro_Trossard  Midfielder  3824.4   7.0
 90       Erling_Haaland    Attacker  6377.1  14.0
 91      Viktor_Gyökeres    Attacker  5828.4   9.0
 92       Benjamin_Sesko    Attacker  4458.6   7.5,
                 Player    Position     Elo  Cost
 1          André_Onana      Keeper  3193.8   5.0
 13     Virgil_van_Dijk    Defender  2747.5   6.0
 14      Joško_Gvardiol    Defender  2657.2   6.0
 15        Fabian_Schär    Defender  2470.9   5.5
 53         Cole_Palmer  Midfielder  3831.7  10.5
 55     Martin_Ødegaard  Midfielder  3592.6   8.0
 56  Dominik_Szoboszlai  Midfielder  

In [169]:
select_top_n_squads("elo_ratings.csv", budget=100, top_n=3, attacker_count=4, midfielder_count=6, defender_count=4, keeper_count=2)


Squad 1:
Total Cost: 100.0 million
Total Elo: 59469.399999999994
                   Player    Position     Elo  Cost
5         Freddie_Woodman      Keeper  3026.3   4.0
6    Giorgi_Mamardashvili      Keeper  2993.2   4.5
10        Maxim_De_Cuyper    Defender  3313.1   4.5
11     Daniel_Muñoz_Mejía    Defender  3281.3   5.5
12    Pedro_Porro_Sauceda    Defender  2940.5   5.5
24         Arthur_Masuaku    Defender  2383.1   4.0
50             Cody_Gakpo  Midfielder  4700.3   7.5
51            Bukayo_Saka  Midfielder  4458.3  10.0
52          Florian_Wirtz  Midfielder  4108.1   8.5
54       Leandro_Trossard  Midfielder  3824.4   7.0
56     Dominik_Szoboszlai  Midfielder  3541.1   6.5
62           Matt_O'Riley  Midfielder  3157.2   5.5
91        Viktor_Gyökeres    Attacker  5828.4   9.0
92         Benjamin_Sesko    Attacker  4458.6   7.5
94             Joël_Piroe    Attacker  3951.5   5.5
109          Zeki_Amdouni    Attacker  3504.0   5.0

Squad 2:
Total Cost: 100.0 million
Total Elo: 533

[                   Player    Position     Elo  Cost
 5         Freddie_Woodman      Keeper  3026.3   4.0
 6    Giorgi_Mamardashvili      Keeper  2993.2   4.5
 10        Maxim_De_Cuyper    Defender  3313.1   4.5
 11     Daniel_Muñoz_Mejía    Defender  3281.3   5.5
 12    Pedro_Porro_Sauceda    Defender  2940.5   5.5
 24         Arthur_Masuaku    Defender  2383.1   4.0
 50             Cody_Gakpo  Midfielder  4700.3   7.5
 51            Bukayo_Saka  Midfielder  4458.3  10.0
 52          Florian_Wirtz  Midfielder  4108.1   8.5
 54       Leandro_Trossard  Midfielder  3824.4   7.0
 56     Dominik_Szoboszlai  Midfielder  3541.1   6.5
 62           Matt_O'Riley  Midfielder  3157.2   5.5
 91        Viktor_Gyökeres    Attacker  5828.4   9.0
 92         Benjamin_Sesko    Attacker  4458.6   7.5
 94             Joël_Piroe    Attacker  3951.5   5.5
 109          Zeki_Amdouni    Attacker  3504.0   5.0,
                                       Player    Position     Elo  Cost
 0                        

In [46]:
from datetime import datetime
async def get_current_gw_player_details():
    async with aiohttp.ClientSession() as session:
        fpl = FPL(session)
        gameweeks = await fpl.get_gameweeks()
        current_gw = next(gw for gw in gameweeks if gw.is_current)
        gw_id = current_gw.id

        players = await fpl.get_players()
        rows = []
        for player in players:
            history = await fpl.get_player_summary(player.id)
            gw_entry = None
            for g in history.history:
                if 'event' in g and g['event'] == gw_id:
                    gw_entry = g
                    break
                elif 'round' in g and g['round'] == gw_id:
                    gw_entry = g
                    break
            if gw_entry:
                rows.append({
                    "Player": f"{player.first_name} {player.second_name}",
                    "Date": gw_entry.get('kickoff_time', None),
                    "Comp": "Premier League",
                    "Round": gw_entry.get('event', gw_entry.get('round', None)),
                    "Opponent": gw_entry.get('opponent_team', None),
                    "Result": gw_entry.get('result', None),
                    "Min": gw_entry.get('minutes', None),
                    "Gls": gw_entry.get('goals_scored', None),
                    "Ast": gw_entry.get('assists', None),
                    'was_home': gw_entry.get('was_home', None),
                    'team_a_score': gw_entry.get('team_a_score', None),
                    'team_h_score': gw_entry.get('team_h_score', None)
                })
        df = pd.DataFrame(rows)
        # Convert 'Date' column from ISO format with time to just date (YYYY-MM-DD)
        df['Date'] = df['Date'].apply(lambda x: datetime.fromisoformat(x.replace("Z", "")).date() if isinstance(x, str) else x)
        return df

# Example usage:
df = await get_current_gw_player_details()
df.to_csv("current_gw_player_details.csv", index=False)
print(df.head())


                         Player        Date            Comp  Round  Opponent  \
0             David Raya Martín  2025-08-17  Premier League      1        14   
1    Kepa Arrizabalaga Revuelta  2025-08-17  Premier League      1        14   
2                     Karl Hein  2025-08-17  Premier League      1        14   
3                 Tommy Setford  2025-08-17  Premier League      1        14   
4  Gabriel dos Santos Magalhães  2025-08-17  Premier League      1        14   

  Result  Min  Gls  Ast  was_home  team_a_score  team_h_score  
0   None   90    0    0     False             1             0  
1   None    0    0    0     False             1             0  
2   None    0    0    0     False             1             0  
3   None    0    0    0     False             1             0  
4   None   90    0    0     False             1             0  


In [49]:
df['was_home'].head()

0    False
1    False
2    False
3    False
4    False
Name: was_home, dtype: bool

In [50]:
df['Season'] = '2025-2026'
df['Comp'] = 'Premier League'
df['Result'] = df.apply(lambda x: f"{x['team_h_score']}-{x['team_a_score']}" if x['was_home'] else f"{x['team_a_score']}-{x['team_h_score']}", axis=1)
df['Pos'] = None


In [59]:
df['Elo'] = 0

In [68]:
for _, row in df.iterrows():
    player_name = row['Player']
    player_file = f"./elo_data/{player_name.replace(' ', '_')}.csv"
    
    new_player_data = df[df['Player'] == player_name].copy()
    # Only select columns that exist in df
    columns_to_select = [col for col in ['Season', 'Date','Comp', 'Round', 'Opponent', 'Result', 'Pos', 'Min', 'Gls', 'Ast', 'Points', 'Elo'] if col in df.columns]
    new_player_data = new_player_data[columns_to_select]
    
    if os.path.exists(player_file):
        player_df = pd.read_csv(player_file)
        # Process player_df as needed
        # print(f"Processing {player_name} from {player_file}")
        player_df2 = pd.concat([player_df, new_player_data], ignore_index=True)
        player_df2.drop_duplicates(subset = ['Season', 'Date','Comp', 'Round', 'Opponent', 'Result', 'Pos', 'Min', 'Gls', 'Ast', 'Points', 'Elo'], keep='last', inplace=True)
        print(player_df2.head())
        # Overwrite the old file with the updated DataFrame
        player_df2.to_csv(player_file, index=False)
    else:
        print(f"File for {player_name} does not exist.")

      Season        Date            Comp        Round        Opponent Result  \
0  2022-2023  2022-08-07  Premier League  Matchweek 1  Leicester City  D 2–2   
1  2022-2023  2022-08-13  Premier League  Matchweek 2  Manchester Utd  W 4–0   
2  2022-2023  2022-08-20  Premier League  Matchweek 3          Fulham  L 2–3   
3  2022-2023  2022-08-27  Premier League  Matchweek 4         Everton  D 1–1   
4  2022-2023  2022-08-30  Premier League  Matchweek 5  Crystal Palace  D 1–1   

  Pos  Min  Gls  Ast  Points     Elo  
0  GK   90    0    0     1.0  1200.0  
1  GK   90    0    0     5.0  1198.8  
2  GK   90    0    0     2.0  1277.6  
3  GK   90    0    0     2.0  1292.4  
4  GK   90    0    0     1.0  1306.6  
      Season        Date            Comp         Round          Opponent  \
0  2022-2023  2022-09-06    Champions Lg   Group stage  hr Dinamo Zagreb   
1  2022-2023  2022-09-14    Champions Lg   Group stage    at RB Salzburg   
2  2022-2023  2022-10-01  Premier League   Matchweek 9   

In [57]:
import os
players = []
for file in os.listdir('./elo_data'):
    if file.endswith('.csv'):
        player_name = file[:-4]
        players.append(player_name)
        with open(os.path.join('./elo_data', file), 'r') as f:
            da
elo_data = []

Aaron_Ramsdale
Aaron_Wan-Bissaka
Aarón_Anselmino
Abdallah_Sima
Abdukodir_Khusanov
Adama_Traoré_Diarra
Adam_Aznou
Adam_Smith
Adam_Webster
Adam_Wharton
Adrian_Mazilu
Adrien_Truffert
Ahmed_Abdullahi
Aji_Alese
Alan_Browne
Albert_Sambi_Lokonga
Alejandro_Garnacho_Ferreyra
Alexander_Isak
Alexis_Mac_Allister
Alex_Cairns
Alex_Iwobi
Alex_Paulsen
Alex_Scott
Alfie_Pond
Alisson_Becker
Alphonse_Areola
Altay_Bayındır
Amadou_Onana
Amad_Diallo
Amara_Nallo
Andreas_Hoelgebaum_Pereira
Andrew_Moran
Andrew_Robertson
Andrey_Nascimento_dos_Santos
André_Onana
André_Trindade_da_Costa_Neto
Andy_Irving
Angus_Gunn
Anthony_Elanga
Anthony_Gordon
Anthony_Patterson
Antoine_Semenyo
Antonee_Robinson
Antoni_Milambo
Antony_dos_Santos
Anton_Stach
Archie_Gray
Armando_Broja
Arthur_Masuaku
Asher_Agbinone
Ashley_Barnes
Axel_Tuanzebe
Ayden_Heaven
Bafodé_Diakité
Bart_Verbruggen
Bashir_Humphreys
Benjamin_Lecomte
Benjamin_Sesko
Benjamin_White
Benoît_Badiashile_Mukinayi
Ben_Broggio
Ben_Davies
Ben_Doak
Ben_Winterburn
Bernardo_Mota_V

In [30]:
dfi = pd.read_csv("initial_elo.csv")
dfw = pd.read_csv("current_gw_player_details.csv")
dfw['Player'] = dfw['Player'].apply(lambda x: x.replace(" ", "_"))

In [31]:
dfi.head()

,Player,Position,Elo,Cost
0,Viktor_Gyökeres,Attacker,2891.2,9.0
1,Mohamed_Salah,Midfielder,2456.6,14.5
2,Erling_Haaland,Attacker,1500.0,14.0
3,Alexander_Isak,Attacker,1500.0,10.5
4,Eberechi_Eze,Midfielder,1947.6,7.5


In [32]:
dfw.head()

,Player,Date,Comp,Round,Opponent,Result,Min,Gls,Ast
0,David_Raya_Martín,2025-08-17T15:30:00Z,Premier League,1,14,NaN,90,0,0
1,Kepa_Arrizabalaga_Revuelta,2025-08-17T15:30:00Z,Premier League,1,14,NaN,0,0,0
2,Karl_Hein,2025-08-17T15:30:00Z,Premier League,1,14,NaN,0,0,0
3,Tommy_Setford,2025-08-17T15:30:00Z,Premier League,1,14,NaN,0,0,0
4,Gabriel_dos_Santos_Magalhães,2025-08-17T15:30:00Z,Premier League,1,14,NaN,90,0,0


In [26]:
"David Raya Martín".replace(" ", "_")	

'David_Raya_Martín'

In [33]:
df_work = dfw.merge(dfi, on='Player', how='left')
df_work = df_work[df_work['Position'].isna()]

In [34]:
df_work.to_csv("df_work.csv", index=False)